In [ ]:
import pandas as pd
from IPython.display import display, HTML
from langchain_openai import ChatOpenAI
from tqdm.auto import tqdm
import dotenv
import os

dotenv.load_dotenv()

In [65]:
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="google/gemini-2.5-flash", 
    temperature=0.0,
)

def count_tokens(text_list: list[str]) -> int:
    total_tokens = 0
    for text in text_list:
        total_tokens += llm.get_num_tokens(text)
    return total_tokens

In [66]:
df = pd.read_csv("../result/viz_df.csv")
df["cluster_label"] = df["cluster_label"].astype(str)
clusters = df.groupby("cluster_label")

In [67]:
df.head(4)

,id,slang,description,umap_x,umap_y,cluster,cluster_prob,cluster_label
0,de7bed5e-22b7-516e-abdd-b6702d14f2c2,W,Shorthand for win,-1.586983,6.195272,29,1.000000,Cluster 29
1,d0f78763-dafa-50e6-98cf-80e37b9ff0ac,L,Shorthand for loss/losing,5.152213,6.530412,7,0.534892,Cluster 7
2,acc7c7d5-9aaa-5d7d-9176-c8963f4ed424,L+ratio,Response to a comment or action on the interne...,5.337737,6.624958,7,0.494982,Cluster 7
3,99ca01c1-e02d-5c30-93f2-843624ab74bd,Dank,excellent or of very high quality,2.778757,6.843239,26,0.970819,Cluster 26


In [68]:
cluster_prompt = """
# title: Thematic Analysis of Slang Terms
## Your Task:
You will analyze slang terms and their descriptions from a single semantic cluster.

Use internal step-by-step reasoning (Chain of Thought) but DO NOT reveal it.
Output only the final concise summary.

Provide the following sections:

1. Theme Title (3–5 words)
2. Theme Description (1–2 sentences)
3. Usage Contexts (1 sentence)
4. Linguistic Traits (1 sentence)
5. Lexical Analysis (word formation, semantic shift, register — 1–2 sentences)
6. Distinctiveness (what makes this cluster unique — 1 sentence)

### Example:
<Example Input>
slay: used to praise someone for doing something impressively
ate: means someone performed extremely well
serve: used to express someone delivering strong style or attitude
mother: used online to praise a female celebrity or creator
queen: praise term for a powerful or admirable woman
<Example Output/>

<Example Output>
Theme Title: Performance & Empowerment Praise
Theme Description: This cluster includes expressive praise terms used to celebrate impressive actions or strong personal style. The tone is highly positive and tied to digital culture.
Usage Contexts: Common in social media comments, fan communities, and casual praise among peers.
Linguistic Traits: Hyperbolic tone, short expressive words, and strong positive emotion.
Lexical Analysis: Includes monosyllabic or simple forms (“slay,” “ate”), metaphorical semantic shifts (“mother”), and identity-marking honorifics (“queen”). Many terms come from AAVE and LGBTQ+ ballroom culture.
Distinctiveness: Focuses on amplified celebration and empowerment tied to performance and style.
<Example Output/>

Follow the same structure for the cluster below.

## Cluster Label:
{cluster_label}

## Cluster Text:
{cluster_text}
"""


### 0) Test the judgement model

In [69]:
# test

cluster_summaries = {}
label = "Cluster 0"
group = clusters.get_group(label)

combined_text = "\n".join([
    f"{row['slang']}: {row['description']}"
    for _, row in group.iterrows()
])

summary = llm.invoke(
    cluster_prompt.format(
        cluster_label=label,
        cluster_text=combined_text
    )
)

cluster_summaries[label] = summary


In [70]:
from pprint import pprint

print("Input:")
pprint(combined_text)

print("\nSummary:")
pprint(summary.content)

Input:
('ILY: I love you\n'
 'ILU: ILU: I Love You\n'
 'ILY :  I love you\n'
 '<3: sideways heart\n'
 '</3: broken heart\n'
 '<3333: a bigger heart\n'
 '143: I love you\n'
 '1432: I love you too\n'
 '459: I love you \n'
 '831: I love you\n'
 'AML: All my love\n'
 'ICFILWU: I could fall in love with you\n'
 'ILU: I love you\n'
 'ILUM: I love you man\n'
 'ILYSM: I love you so much\n'
 'ILY: I love you\n'
 'ImL:  I love you\n'
 'ISLY: I still love you\n'
 'IWALU: I will always love you\n'
 'IYQ: I like you\n'
 'LY: Love ya\n'
 'LYLAS: Love you like a sis\n'
 'LYLC: Love you like crazy\n'
 'LYSM: Love you so much')

Summary:
('Theme Title: Abbreviated Expressions of Love\n'
 'Theme Description: This cluster encompasses various shorthand forms, '
 'acronyms, and numerical codes used to convey feelings of love, affection, '
 'and emotional connection. These terms facilitate quick and intimate '
 'communication of romantic or platonic love.\n'
 'Usage Contexts: Primarily used in informal digi

### 1) Run thematic analysis on each cluster, gather summarize.

In [71]:
cluster_summaries = {}

for label, group in tqdm(clusters, desc="Processing Clusters"):
    # Combine slang + description for the cluster
    combined_text = "\n".join([
        f"{row['slang']}: {row['description']}"
        for _, row in group.iterrows()
    ])

    summary = llm.invoke(
        cluster_prompt.format(
            cluster_label=label,
            cluster_text=combined_text
        )
    )

    cluster_summaries[label] = summary


Processing Clusters:   0%|          | 0/31 [00:00<?, ?it/s]

In [72]:
all_summary_text = "\n\n".join(
    [f"{label}:\n{summary}" for label, summary in cluster_summaries.items()]
)

Show cluster summarize texts

In [73]:
df_summaries = pd.DataFrame([
    {"cluster_label": label, "summary": summary.content}  # first 150 chars
    for label, summary in cluster_summaries.items()
])

In [76]:
df_summaries = df_summaries.sort_values("cluster_label").reset_index(drop=True)

display(HTML(df_summaries.to_html().replace(
    '<table', '<table style="font-size:12px; font-family:Arial; line-height:1.5;"'
)))

,cluster_label,summary
0,Cluster 0,"Theme Title: Affectionate Digital Shorthand\nTheme Description: This cluster comprises various abbreviations and symbols used to express love and affection in digital communication. It reflects a desire for quick, intimate expressions of care.\nUsage Contexts: Primarily found in text messages, instant messaging, and social media comments between close individuals.\nLinguistic Traits: Predominantly initialisms, numerical codes, and emoticons, often conveying strong positive emotion.\nLexical Analysis: Features extensive use of acronyms and initialisms (ILY, LYSM), numerical substitutions (143), and symbolic representations (<3). These forms prioritize brevity and informal register.\nDistinctiveness: Characterized by its high density of direct ""I love you"" variations and numerical/symbolic codes for expressing affection."
1,Cluster 1,"Theme Title: Exclamations & Online Shorthand\nTheme Description: This cluster comprises acronyms and initialisms primarily used for expressing surprise, frustration, or affirmation, often in digital communication. It reflects a need for quick, informal expression.\nUsage Contexts: Predominantly found in online chats, text messages, and social media.\nLinguistic Traits: Heavy reliance on initialisms and acronyms, often representing common exclamatory phrases.\nLexical Analysis: Word formation is dominated by acronyms and initialisms, with some phonetic spellings (""OHHEMMGEE"") and abbreviations of common phrases (""On god""). Semantic shifts are minimal, largely retaining the original phrase's meaning.\nDistinctiveness: Characterized by its high density of initialisms and abbreviations for expressing immediate emotional reactions or brief status updates."
2,Cluster 10,"Theme Title: Acronyms, Initialisms & Digital Shorthand\nTheme Description: This cluster primarily consists of abbreviations and initialisms used for rapid communication, often in digital contexts. It also includes a few full slang terms.\nUsage Contexts: Predominantly found in online chats, text messages, gaming, and social media.\nLinguistic Traits: Heavy reliance on acronyms and initialisms, often phonetic spellings, and some portmanteaus.\nLexical Analysis: Word formation is dominated by initialisms and acronyms, with some phonetic spellings (""prolly,"" ""pwn"") and semantic shifts (""pick me girl""). The register is highly informal and context-dependent.\nDistinctiveness: Characterized by its extensive use of abbreviations for efficiency, particularly in digital communication, and a focus on parental surveillance."
3,Cluster 11,"Theme Title: Abbreviated Online Communication\nTheme Description: This cluster comprises acronyms and initialisms predominantly used for quick communication in digital environments. They facilitate brevity and convey specific questions, statements, or emotional responses.\nUsage Contexts: Primarily found in online chat, forums, gaming, and text messaging.\nLinguistic Traits: Heavy reliance on initialisms and phonetic abbreviations, often representing full phrases or concepts.\nLexical Analysis: Dominated by acronyms and initialisms (FAQ, QFT), with some phonetic spellings (QIK, Q_Q) and borrowings (QPSA?). Semantic shifts are minimal, focusing on abbreviation for efficiency.\nDistinctiveness: Characterized by its high density of 'Q' initialisms and its primary function of streamlining digital dialogue."
4,Cluster 12,"Theme Title: Acronyms and Parental References\nTheme Description: This cluster primarily consists of acronyms and initialisms, alongside terms related to parental figures or family roles. It reflects a mix of digital shorthand and familial associations.\nUsage Contexts: Predominantly found in informal digital communication, text messages, and online forums.\nLinguistic Traits: High prevalence of initialisms and abbreviations, often representing common phrases or roles.\nLexical Analysis: Word formation relies heavily on acronyms and initialisms (e.g., ""

In [ ]:
# count total output tokens
print(f"Total output tokens from cluster summaries: {count_tokens(list(df_summaries['summary']))}")

Total output tokens from cluster summaries: 5502


### 2) Run thematic analysis on all cluster, gain overall analysis.

In [82]:
llm_candidates = [
    ChatOpenAI(
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        model=model_name,
        temperature=0.0,
    )
    for model_name in [
        "google/gemini-2.5-flash",
        "x-ai/grok-4.1-fast:free",
        "openai/gpt-4o-mini",
    ]
]

In [90]:
final_prompt = """
You are given concise summaries of semantic clusters from a slang dataset.

Use internal reasoning (Chain of Thought) but DO NOT reveal it. 
Only output your final analysis.

Your task is to **summarize each cluster** by giving:

1. A **unique theme name** for the cluster (3–5 words)
2. A **short description** (1–2 sentences) explaining the cluster's semantic focus and social/cultural context

Do NOT create new overarching themes or merge clusters.  
Keep the output structured, concise, and readable.  

Format example:

Cluster 0: Theme Name: <theme title>
Description: <short description>

Cluster 1: Theme Name: <theme title>
Description: <short description>

...

### Cluster Summaries:
{all_summaries}
"""


In [91]:
final_thematic_analysis = {}

for llm in tqdm(llm_candidates):
    try:
        final_thematic_analysis[llm.model_name] = llm.invoke(
            final_prompt.format(
                all_summaries=all_summary_text
            )
        )
        print(f"Model {llm.model_name} succeeded.")

    except Exception as e:
        print(f"Model {llm.model_name} failed with error: {e}")
        continue

  0%|          | 0/3 [00:00<?, ?it/s]

Model google/gemini-2.5-flash succeeded.
Model x-ai/grok-4.1-fast:free succeeded.
Model openai/gpt-4o-mini succeeded.


In [92]:
df_thematic = pd.DataFrame([
    {"model_name": model, "thematic_analysis": text}
    for model, text in final_thematic_analysis.items()
])

display(HTML(df_thematic.to_html().replace(
    '<table', '<table style="font-size:12px; font-family:Arial; line-height:1.2;"'
)))

In [97]:
pprint(df_thematic['thematic_analysis'][0].content)

('Cluster 0: Theme Name: Affectionate Digital Shorthand\n'
 'Description: This cluster comprises abbreviations and symbols used to '
 'express love and affection in digital communication, reflecting a desire for '
 'quick, intimate expressions of care.\n'
 '\n'
 'Cluster 1: Theme Name: Exclamations & Online Shorthand\n'
 'Description: This cluster comprises acronyms and initialisms primarily used '
 'for expressing surprise, frustration, or affirmation in digital '
 'communication, reflecting a need for quick, informal expression.\n'
 '\n'
 'Cluster 2: Theme Name: Abbreviated Digital Communication\n'
 'Description: This cluster comprises a wide array of abbreviations and '
 'acronyms primarily used for quick, informal communication in digital '
 'environments, conveying various states, actions, and conversational cues.\n'
 '\n'
 'Cluster 3: Theme Name: Keyboard-Centric Status Updates\n'
 'Description: This cluster comprises acronyms and abbreviations indicating a '
 "user's presence, a

In [98]:
pprint(df_thematic['thematic_analysis'][1].content)

('Cluster 0: Theme Name: Loving Text Abbreviations\n'
 'Description: Short forms like ILY and 143 express affection in digital '
 'messages, fostering intimate bonds in casual online interactions among '
 'friends and partners.\n'
 '\n'
 'Cluster 1: Theme Name: Emotional Exclamation Shorthand\n'
 'Description: Acronyms for surprise or affirmation in chats reflect '
 'fast-paced digital exchanges, common in social media for quick emotional '
 'reactions.\n'
 '\n'
 'Cluster 2: Theme Name: O-Prefixed Chat Shortcuts\n'
 'Description: Brevity-focused abbreviations starting with O denote actions or '
 'hugs in online forums, emphasizing efficiency in informal digital '
 'conversations.\n'
 '\n'
 'Cluster 3: Theme Name: Keyboard Status Indicators\n'
 'Description: Acronyms like AFK signal user availability near computers, '
 'rooted in gaming and chat room culture for real-time coordination.\n'
 '\n'
 'Cluster 4: Theme Name: General Digital Acronyms\n'
 'Description: Varied abbreviations shor

In [99]:
pprint(df_thematic['thematic_analysis'][2].content)

('Cluster 0: Theme Name: Affectionate Digital Shorthand  \n'
 'Description: This cluster comprises various abbreviations and symbols used '
 'to express love and affection in digital communication, reflecting a desire '
 'for quick, intimate expressions of care.\n'
 '\n'
 'Cluster 1: Theme Name: Exclamations & Online Shorthand  \n'
 'Description: This cluster consists of acronyms and initialisms primarily '
 'used for expressing surprise, frustration, or affirmation in digital '
 'communication, highlighting the need for quick, informal expression.\n'
 '\n'
 'Cluster 10: Theme Name: Acronyms, Initialisms & Digital Shorthand  \n'
 'Description: This cluster includes abbreviations and initialisms used for '
 'rapid communication in digital contexts, emphasizing efficiency and informal '
 'language.\n'
 '\n'
 'Cluster 11: Theme Name: Abbreviated Online Communication  \n'
 'Description: This cluster features acronyms and initialisms for quick '
 'communication in digital environments, faci